# Testing model infrastructures 

# **IMPORT LIBRARY**

In [ ]:
#pip install tf-models-officials
!pip install -q tensorflow-recommenders 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 3.5 MB/s eta 0:00:00


In [ ]:
#perhitungan
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# PreProcessing
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, StandardScaler, OneHotEncoder, MinMaxScaler
from typing import Dict, Text

# tensorflow library
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs # riweuh instalnya, bingung

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Data Cleaning**

**Note**: form-response.csv adalah data dari form yang sudah dimodifikasi bagian header untuk mempermudah navigasi. Data Sudah di upload di Gdrive Bersama

In [ ]:
form_data = pd.read_csv('form-response.csv')
form_data.columns

Index(['Timestamp', 'Email Address', 'Nama Tempat Ibadah',
       'Jenis Tempat Ibadah', 'Lokasi Tempat Ibadah', 'donasi',
       'donasi_detail', 'Keamanan', 'rating', 'rating_review'],
      dtype='object')

Deteksi Invalid Data, lalu buang row tersebut dan Cari awalan "MASJID/MESJID", "GEREJA", DSB dan buang kata tersebut

In [ ]:
form_data['Nama Tempat Ibadah'] = form_data['Nama Tempat Ibadah'].apply(lambda x: x.upper())
form_data['Lokasi Tempat Ibadah'] = form_data['Lokasi Tempat Ibadah'].apply(lambda x: x.title().rstrip().lstrip())
form_data['Jenis Tempat Ibadah'] =form_data['Jenis Tempat Ibadah'].apply(lambda x: x.lower())
extracted = form_data['Nama Tempat Ibadah'].str.extract(r'^(GA|PO)')
extracted_idx = extracted[extracted[0] == "GA"].index.values

In [ ]:
form_data.loc[extracted_idx]

,Timestamp,Email Address,Nama Tempat Ibadah,Jenis Tempat Ibadah,Lokasi Tempat Ibadah,donasi,donasi_detail,Keamanan,rating,rating_review
3,14/05/2023 14:53:45,haekalfadah@gmail.com,GATAU NAMANYA;(,masjid,Surabaya,Iya,tidak mengetahui alokasi dana tersebut kemana,cukup aman karena banyak warga dan pintu selal...,5,kedepannya dipertingkatkan lagi panitia remaja...
46,17/05/2023 0:20:56,NaN,GATAU POKONYA MASJID NU BESAR DI SUTOREJO,masjid,Sutorejo,Iya,Anak yatim,"Cukup aman, terkadang ada penjaganya di gerbang",4,Untuk ke tempat ibadah bagian wanita sebaiknya...


In [ ]:
extracted_repair = form_data['Nama Tempat Ibadah'].str.extract(r'(\b(MA|ME)S?JID\b|\bGEREJA\b)')
extracted_repair

,0,1
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,MASJID,MA
...,...,...
114,NaN,NaN
115,GEREJA,NaN
116,NaN,NaN
117,GEREJA,NaN


In [ ]:
extracted_repair_idx = np.where(extracted_repair[0].notnull() == True)
extracted_repair_idx[0]

array([  4,   5,   6,  10,  11,  12,  13,  15,  17,  18,  20,  21,  23,
        24,  25,  26,  27,  28,  29,  30,  31,  33,  34,  35,  36,  38,
        39,  40,  41,  43,  44,  45,  46,  47,  48,  49,  51,  52,  53,
        55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,
        69,  71,  72,  73,  74,  76,  77,  78,  80,  81,  82,  83,  85,
        87,  88,  89,  90,  91,  93,  94,  96,  98, 100, 105, 106, 107,
       108, 109, 112, 113, 115, 117, 118])

In [ ]:
form_data.loc[extracted_repair_idx[0],'Nama Tempat Ibadah'] = form_data.loc[extracted_repair_idx[0],'Nama Tempat Ibadah'].apply(lambda x: x.partition(" ")[2])
form_data = form_data.drop(extracted_idx, axis=0)

In [ ]:
print(len(form_data))
form_data

117


,Timestamp,Email Address,Nama Tempat Ibadah,Jenis Tempat Ibadah,Lokasi Tempat Ibadah,donasi,donasi_detail,Keamanan,rating,rating_review
0,14/05/2023 14:40:58,rosiindah634@gmail.com,NURUL HUDA,masjid,Kebumen,Iya,buat memperbaiki infrastruktur,Aman karena di desa,5,karena masjid tersebut dirawat bersama sama ol...
1,14/05/2023 14:43:08,gregoryhutauruk1@gmail.com,HKBP TEBET,gereja,Jakarta Timur,Iya,"Untuk didonasikan ke panti asuhan, korban benc...",Sanat aman,5,Sangat nyaman untuk beribadah
2,14/05/2023 14:43:10,reninuragustin94@gmail.com,ULUL AZMI,masjid,Surabaya,Iya,"Untuk anak panti, nasi gratis Jum'at, dll","Aman, namun perlu dilakukan kunci ganda untuk ...",4,Ada pintu toilet yang rusak dan toilet terkada...
4,14/05/2023 15:03:54,diasayu2307@gmail.com,ULUL AZMI,masjid,Surabaya,Iya,saya tidak tahu dan tidak pernah menanyakan juga,"menurut saya kurang, meskipun ada CCTV ternyat...",4,"Tempat wudhu, toilet dan area sholat bersih. T..."
5,14/05/2023 15:11:51,andre.bima.saktiawan-2020@fisip.unair.ac.id,NURUL HUDA,masjid,Surabaya,Iya,Tidak,Sangat baik. Meskipun berada di perumahan pada...,4,"Cukup bersih, karena sudah ada panitia masjid ..."
...,...,...,...,...,...,...,...,...,...,...
114,NaN,NaN,GBI FILADELFIA SAWAHAN,gereja,Surabaya,-,-,-,4,no review
115,NaN,NaN,KRISTEN KALAM KUDUS KUPANG JAYA,gereja,Surabaya,-,-,-,5,Gereja yang memiliki kekuatan dalam kasih pers...
116,NaN,NaN,GMAHK JEMAAT TIDAR II,gereja,Surabaya,-,-,-,4,Gereja yg sangat sederhana tapi bersih tmpatny...
117,NaN,NaN,HAPPY FAMILY CENTER RICH PALACE,gereja,Surabaya,-,-,-,5,Anointed Church with super good people communi...


Membuat 2 buah dataset baru, yaitu:
1.   Rekomendasi Berdasarkan rating \(Arsitektur Menggunakan Dense Layer)
2.   Rekomendasi Berdasarkan Review \(Arsitektur Menggunakan Word Embedding)


Dataset Berdasarkan Rating

In [ ]:
rating_data = form_data.drop(['Timestamp', 'Email Address', 'donasi', 'donasi_detail', 'Keamanan'], axis=1)
rating_data

,Nama Tempat Ibadah,Jenis Tempat Ibadah,Lokasi Tempat Ibadah,rating,rating_review
0,NURUL HUDA,masjid,Kebumen,5,karena masjid tersebut dirawat bersama sama ol...
1,HKBP TEBET,gereja,Jakarta Timur,5,Sangat nyaman untuk beribadah
2,ULUL AZMI,masjid,Surabaya,4,Ada pintu toilet yang rusak dan toilet terkada...
4,ULUL AZMI,masjid,Surabaya,4,"Tempat wudhu, toilet dan area sholat bersih. T..."
5,NURUL HUDA,masjid,Surabaya,4,"Cukup bersih, karena sudah ada panitia masjid ..."
...,...,...,...,...,...
114,GBI FILADELFIA SAWAHAN,gereja,Surabaya,4,no review
115,KRISTEN KALAM KUDUS KUPANG JAYA,gereja,Surabaya,5,Gereja yang memiliki kekuatan dalam kasih pers...
116,GMAHK JEMAAT TIDAR II,gereja,Surabaya,4,Gereja yg sangat sederhana tapi bersih tmpatny...
117,HAPPY FAMILY CENTER RICH PALACE,gereja,Surabaya,5,Anointed Church with super good people communi...


In [ ]:
# # Save dataset dengan nama 'rating_based.csv'
# # Uncomment Jika Perlu
# rating_data.to_csv('rating_based.csv')

Dataset Berdasarkan Review

In [ ]:
review_data = form_data.drop(['Timestamp', 'Email Address'], axis=1)
review_data

,Nama Tempat Ibadah,Jenis Tempat Ibadah,Lokasi Tempat Ibadah,donasi,donasi_detail,Keamanan,rating,rating_review
0,NURUL HUDA,masjid,Kebumen,Iya,buat memperbaiki infrastruktur,Aman karena di desa,5,karena masjid tersebut dirawat bersama sama ol...
1,HKBP TEBET,gereja,Jakarta Timur,Iya,"Untuk didonasikan ke panti asuhan, korban benc...",Sanat aman,5,Sangat nyaman untuk beribadah
2,ULUL AZMI,masjid,Surabaya,Iya,"Untuk anak panti, nasi gratis Jum'at, dll","Aman, namun perlu dilakukan kunci ganda untuk ...",4,Ada pintu toilet yang rusak dan toilet terkada...
4,ULUL AZMI,masjid,Surabaya,Iya,saya tidak tahu dan tidak pernah menanyakan juga,"menurut saya kurang, meskipun ada CCTV ternyat...",4,"Tempat wudhu, toilet dan area sholat bersih. T..."
5,NURUL HUDA,masjid,Surabaya,Iya,Tidak,Sangat baik. Meskipun berada di perumahan pada...,4,"Cukup bersih, karena sudah ada panitia masjid ..."
...,...,...,...,...,...,...,...,...
114,GBI FILADELFIA SAWAHAN,gereja,Surabaya,-,-,-,4,no review
115,KRISTEN KALAM KUDUS KUPANG JAYA,gereja,Surabaya,-,-,-,5,Gereja yang memiliki kekuatan dalam kasih pers...
116,GMAHK JEMAAT TIDAR II,gereja,Surabaya,-,-,-,4,Gereja yg sangat sederhana tapi bersih tmpatny...
117,HAPPY FAMILY CENTER RICH PALACE,gereja,Surabaya,-,-,-,5,Anointed Church with super good people communi...


In [ ]:
#  # Save dataset dengan nama 'review_based.csv'
#  # Uncomment Jika Perlu
# review_data.to_csv('review_based.csv')

**Note:** Hasil Export akan Muncul beberapa saat di tab files, jadi harap bersabar

# **Data Preprocessing**

Rating Based Data Preprocessing

In [ ]:
rating_data

,Nama Tempat Ibadah,Jenis Tempat Ibadah,Lokasi Tempat Ibadah,rating,rating_review
0,NURUL HUDA,masjid,Kebumen,5,karena masjid tersebut dirawat bersama sama ol...
1,HKBP TEBET,gereja,Jakarta Timur,5,Sangat nyaman untuk beribadah
2,ULUL AZMI,masjid,Surabaya,4,Ada pintu toilet yang rusak dan toilet terkada...
4,ULUL AZMI,masjid,Surabaya,4,"Tempat wudhu, toilet dan area sholat bersih. T..."
5,NURUL HUDA,masjid,Surabaya,4,"Cukup bersih, karena sudah ada panitia masjid ..."
...,...,...,...,...,...
114,GBI FILADELFIA SAWAHAN,gereja,Surabaya,4,no review
115,KRISTEN KALAM KUDUS KUPANG JAYA,gereja,Surabaya,5,Gereja yang memiliki kekuatan dalam kasih pers...
116,GMAHK JEMAAT TIDAR II,gereja,Surabaya,4,Gereja yg sangat sederhana tapi bersih tmpatny...
117,HAPPY FAMILY CENTER RICH PALACE,gereja,Surabaya,5,Anointed Church with super good people communi...


In [ ]:
x_data = rating_data.iloc[:,:-2]
y_data = rating_data.iloc[:,-2:]

In [ ]:
labeled_place = x_data.groupby('Nama Tempat Ibadah').ngroup()
x_data['Nama Tempat Ibadah'] = labeled_place

In [ ]:
le = LabelEncoder()
x_data['Jenis Tempat Ibadah'] = le.fit_transform(x_data['Jenis Tempat Ibadah'])
dict(enumerate(le.classes_,0))

{0: 'gereja',
 1: 'gereja katolik',
 2: 'gereja protestan',
 3: 'masjid',
 4: 'pura'}

In [ ]:
x_data

,Nama Tempat Ibadah,Jenis Tempat Ibadah,Lokasi Tempat Ibadah
0,80,3,Kebumen
1,63,0,Jakarta Timur
2,98,3,Surabaya
4,98,3,Surabaya
5,79,3,Surabaya
...,...,...,...
114,52,0,Surabaya
115,72,0,Surabaya
116,57,0,Surabaya
117,61,0,Surabaya


In [ ]:
# encoder = OneHotEncoder(handle_unknown="ignore", sparse=False)
# ct = ColumnTransformer(transformers=[("onehot", encoder, ['Nama Tempat Ibadah'])], remainder="passthrough")
# col_names = encoder.get_feature_names(['enc', 'Nama Tempat Ibadah', 'Jenis Tempat Ibadah', 'Lokasi Tempat Ibadah', 'rating', 'rating_review'])


In [ ]:
encoded = pd.get_dummies(x_data['Lokasi Tempat Ibadah'])
new_encoded_col_names = dict(enumerate(encoded, 1))
new_encoded_col_names

{1: 'Bandung',
 2: 'Bandung Barat',
 3: 'Banyuwangi',
 4: 'Batujajar',
 5: 'Bekasi',
 6: 'Bogor',
 7: 'Bojonegoro',
 8: 'Ciamis',
 9: 'Cililin',
 10: 'Cimahi',
 11: 'Cipatat',
 12: 'Dayeuhkolot',
 13: 'Demak',
 14: 'Depok',
 15: 'Garut',
 16: 'Gubeng',
 17: 'Jakarta Timur',
 18: 'Jombang',
 19: 'Kebumen',
 20: 'Kediri',
 21: 'Malang',
 22: 'Manyar',
 23: 'Mulyorejo',
 24: 'Purwakarta',
 25: 'Sidoarjo',
 26: 'Sumedang',
 27: 'Surabaya'}

In [ ]:
new_encoded_col_names = {v: k for k, v in new_encoded_col_names.items()}

encoded = encoded.rename(columns=new_encoded_col_names)

In [ ]:
x_encoded = pd.concat([x_data.iloc[:, :-1], encoded], axis=1)
x_encoded

,Nama Tempat Ibadah,Jenis Tempat Ibadah,1,2,3,4,5,6,7,8,...,18,19,20,21,22,23,24,25,26,27
0,80,3,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,63,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,98,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,98,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,79,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,52,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
115,72,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
116,57,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
117,61,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
x_encoded.describe()

,Nama Tempat Ibadah,Jenis Tempat Ibadah,1,2,3,4,5,6,7,8,...,18,19,20,21,22,23,24,25,26,27
count,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,...,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000
mean,49.324786,2.598291,0.162393,0.102564,0.034188,0.017094,0.025641,0.008547,0.051282,0.017094,...,0.008547,0.008547,0.017094,0.017094,0.008547,0.008547,0.017094,0.017094,0.025641,0.367521
std,29.823570,1.000516,0.370397,0.304694,0.182493,0.130179,0.158742,0.092450,0.221521,0.130179,...,0.092450,0.092450,0.130179,0.130179,0.092450,0.092450,0.130179,0.130179,0.158742,0.484204
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,48.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,75.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,99.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


**Note :** Kolom 0 sampai 26 merupakan one-hot code dari kolom **'lokasi tempat ibadah'**

In [ ]:
x_encoded.columns = x_encoded.columns.astype(str)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_encoded, y_data, test_size=0.25)
print(f'Jumlah data train : {len(x_train)}\nJumlah data validasi : {len(x_val)}')

Jumlah data train : 87
Jumlah data validasi : 30


Data Normalization

In [ ]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_val = sc.transform(x_val)

In [ ]:
x_train

array([[-0.56504208,  0.31822291,  2.10653744, ..., -0.153393  ,
        -0.18898224, -0.65286255],
       [-0.33111236,  0.31822291,  2.10653744, ..., -0.153393  ,
        -0.18898224, -0.65286255],
       [ 1.30639573,  1.47178098, -0.47471266, ..., -0.153393  ,
        -0.18898224,  1.53171598],
       ...,
       [-0.56504208,  0.31822291, -0.47471266, ..., -0.153393  ,
        -0.18898224, -0.65286255],
       [ 0.50435095, -3.14245127, -0.47471266, ..., -0.153393  ,
        -0.18898224, -0.65286255],
       [-0.83239034,  0.31822291,  2.10653744, ..., -0.153393  ,
        -0.18898224, -0.65286255]])

# **Model Testing**

# algoritma model testing untuk rekomendasi berdasarkan rating

In [ ]:
from tensorflow import keras

#rekomendasi berdasarkan rating
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(2,)),
        tf.keras.layers.Dense(16, activation='relu', name = 'layer1'),
        tf.keras.layers.Dense(1, activation='sigmoid', name = 'layer2')
     ]
)


In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 16)                48        
                                                                 
 layer2 (Dense)              (None, 1)                 17        
                                                                 
Total params: 65
Trainable params: 65
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model2 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Algoritma model testing untuk rekomendasi berdasarkan review.

1. stopwords. apa aja stopwordsnya => Pake https://pypi.org/project/Sastrawi/
2. grouping reviewnya positif (1) negatif (0)



# **MODEL** **VALIDATION**

# ***MOVIE DATASET***

In [ ]:
# Load dataset ratings
ratings = tfds.load("movielens/100k-ratings", split = "train")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-ratings/0.1.1.incompleteG9580V/movielens-train.tfrecord*...…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-ratings/0.1.1. Subsequent calls will reuse this data.


In [ ]:
# gatau ini apa namanya, pokoknya tujuannya dipisah-pisah lah intinya, ntar ganti sendiri ya
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_rating": x["user_rating"]
})

split data buat kyk yg atas, 75%  train data, 25% test data

In [ ]:
tf.random.set_seed(30) #30 ini acak aja
shuffled = ratings.shuffle(100_000, seed = 30, reshuffle_each_iteration=False) 

train_data = shuffled.take(75_000)
test_data = shuffled.skip(75_000).take(25_000)

In [ ]:
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])
mt_unique = np.unique(np.concatenate(list(movie_titles)))

In [ ]:
# # build model architecture

class ranking_model(tf.keras.Model):
  def __init__(self):  #  init ni aku baru belajar, nyontek google, mohon koreksi kalau salah
    super().__init__()
    embedding_dimension = 24

    # tokenizer
    self.movie_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(vocabulary = mt_unique, mask_token=None),
        tf.keras.layers.Embedding(len(mt_unique) + 1, embedding_dimension)
    ])


    ## prediksi ratings
    self.ratings = tf.keras.Sequential([
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid") #info
        #tf.keras.layers.Lambda(self.check)
    ])

  
  def check (self):
  def callbacks(self, inputs):

    movie_title = inputs

    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([movie_embedding], axis = 1))

In [ ]:
# # lost
task = tfrs.tasks.Ranking(
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = [tf.keras.metrics.BinaryAccuracy()]
)

In [ ]:
from keras.metrics.accuracy_metrics import BinaryAccuracy
from keras.api._v2.keras.losses import BinaryCrossentropy
class movie_model(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = ranking_model()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss = tf.keras.losses.BinaryCrossentropy(),
        metrics = [tf.keras.metrics.BinaryAccuracy()]
    )
  
  def callback(self, feat: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (feat["movie_title"])
    )

  def compute_loss(Self, feat:Dict[Text, tf.Tensor], training = False) -> tf.Tensor:
    labels = feat.pop("user_rating")

    rating_predictions = self(feat)

    return self.task(labels=labels, predictions=rating_predictions)

In [ ]:
model = movie_model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))

In [ ]:
cached_train_data = train_data.shuffle(100_000).batch(8046).cache()
cached_test_data = test_data.batch(3000).cache()
model.fit(cached_train_data, epochs=3)

Epoch 1/3


NameError: ignored

1. User id harus include
2. movie_id > predicted ratings.
3. endingnya outputnya model user dan model movie
4. keep dulu model user 
5. model movie di load, trainable = False. trs di transfer learning pake Dense Layer data kita. layer terakhir unitnya sesuai len data agama database (pake query)
6. load model user + model movie di training functional API
7. trs didapet outputnya,